In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import random
import time
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from data import ChristmasImages
from torchvision.datasets import ImageFolder
from model import Network

#To calculate mean and std_dev of training dataset

path = 'image_data'
train_transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.CenterCrop(192),
            transforms.RandomRotation(degrees = (-180,180)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            #transforms.Normalize(mean= torch.tensor([0.4585, 0.4114, 0.3843]),std = torch.tensor([0.3152, 0.3112, 0.3195])),
            ])
image_data_unnormalized = ImageFolder(root = path + '/train', transform = train_transform)
loader = DataLoader(image_data_unnormalized,batch_size = len(image_data_unnormalized),num_workers = 6)

images,labels = next(iter(loader))

mean,std = images.mean([0,2,3]),images.std([0,2,3])

print (mean,std)
 

In [ ]:
train_path = 'image_data'
train_dataset = ChristmasImages(train_path,training = True)
train_dataset[0]

In [ ]:
rand_idx = random.randrange(0,3725,1)
image,label = dataset[rand_idx]
#print(len(val_data))
print(image,label)
plt.imshow(image.permute(1,2,0))


In [ ]:
batch_size = 32
val_size = int(0.2*len(dataset))
train_size = len(dataset) - val_size
print(val_size,train_size)

In [ ]:
train_ds,val_ds = random_split(dataset,[train_size,val_size])

In [ ]:
train_dl = DataLoader(train_ds, batch_size, shuffle = True, num_workers = 6)
val_dl = DataLoader(val_ds, batch_size,shuffle = True, num_workers = 6)

In [ ]:
from torchsummary import summary 
mymodel = Network()
#new_model.freeze_parameters()
summary(mymodel,(3,224,224))

In [ ]:
learning_rate = 0.001
optimizer = torch.optim.Adam(mymodel.parameters(),lr = 0.001,weight_decay = 0.0001)
#optimizer = torch.optim.SGD(new_model.parameters(), lr = 3e-4,weight_decay = 0.9)
loss_function = nn.CrossEntropyLoss()
num_epochs = 30
patience = 2
min_lr = 3e-4
best_val_acc = 0
no_improvements_epochs = 0

In [ ]:
optimizer = torch.optim.Adam(mymodel.parameters(),lr = 0.01,weight_decay = 0.0001)
best_val_acc = 0
num_epochs = 10

In [ ]:
for epoch in range(num_epochs):
    total_train_loss = 0
    train_correct = 0
    train_total = 0
    
    time_start = time.time()
    with tqdm(train_dl, unit = "batch", desc = f"Epoch {epoch+1}") as tepoch:
        # Training step
        trained_model.train()
        for data, target in tepoch:
            optimizer.zero_grad()
            output = trained_model(data)
            loss = loss_function(output, target)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            train_total += target.size(0)
            train_correct += (predicted == target).sum().item()
        train_acc = train_correct / train_total
        train_loss = total_train_loss / len(train_dl)
        tepoch.set_postfix(loss = train_loss, acc = train_acc)
        
    # Evaluation step
    with torch.no_grad():
        trained_model.eval()
        total_val_loss = 0
        val_correct = 0
        val_total = 0
        for data, target in val_dl:
            output = trained_model(data)
            val_loss = loss_function(output, target)
            total_val_loss += val_loss.item()
            _, predicted = torch.max(output.data, 1)
            val_total += target.size(0)
            val_correct += (predicted == target).sum().item()
        val_acc = val_correct / val_total
        val_loss = total_val_loss / len(val_dl)
        if val_acc>best_val_acc:
            best_val_acc = val_acc
            best_weights = trained_model.state_dict()
            torch.save(best_weights, 'best_weights.pth')
            trained_model.save_model()
 
        
    time_end = time.time()
    time_elapsed = time_end - time_start
    print("Time_elapsed for Epoch [{}] : [{:.2f}] s".format(epoch+1,time_elapsed))
    print(f'Training Loss: {train_loss:.4f} | Train Acc: {100*train_acc:.4f}')
    print(f'Validation Loss: {val_loss:.4f} | Validation Acc: {100*val_acc:.4f} | Best Validation Acc: {100*best_val_acc:.4f}')
        

In [ ]:
trained_model = Network()
trained_model.load_state_dict(torch.load('model'))